# Mosaico / Anatel

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/intrig-unicamp/hackathon5G/blob/main/datasets/mosaico.ipynb)

Esse Notebook contém uma breve análise dos dados que o Mosaico dispõe. Os participantes são encorajados a experimentar com os dados para descobrir padrões, formas de resolver problemas no contexto da Hackathon.

O Mosaico é um sistema da Agência Nacional de Telecomunicações (Anatel) que contém informações sobre todas as estações de telecomunicações registradas no Brasil. O sistema é utilizado para monitorar e controlar o espectro de frequências utilizado pelas operadoras de telecomunicações e outras empresas que possuem licenças para operar estações em território nacional. Entre os dados armazenados no Mosaico estão informações sobre os proprietários das estações, as tecnologias e equipamentos utilizados, as frequências de transmissão e recepção, a localização geográfica das estações e as datas de licenciamento e validade. O Mosaico é uma ferramenta importante para a ANATEL e outras autoridades regulatórias que precisam monitorar e controlar o uso do espectro de frequências de telecomunicações no Brasil, garantindo que as operadoras e outras empresas cumpram as regulamentações e leis aplicáveis.

Os dados utilizados nesse Notebook foram extraídos [desse](https://sistemas.anatel.gov.br/se/public/view/b/licenciamento.php?view=licenciamento) endereço (Mosaico).

## Instalação das dependências

Instalar as dependências no ambiente adequado (pip ou Anaconda).

In [6]:
try:
    import google.colab as _
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    !pip install pandas numpy seaborn scikit-learn plotly tqdm -q
else:
    !conda install pandas numpy seaborn scikit-learn plotly tqdm -qy --freeze-installed

Solving environment: ...working... done

## Package Plan ##

  environment location: /home/arthur/miniconda3

  added / updated specs:
    - numpy
    - pandas
    - plotly
    - scikit-learn
    - seaborn
    - tqdm


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    plotly-5.14.1              |     pyhd8ed1ab_0         4.9 MB  conda-forge
    scikit-learn-1.2.2         |   py39hd189fd4_1         7.3 MB  conda-forge
    ------------------------------------------------------------
                                           Total:        12.1 MB

The following packages will be UPDATED:

  plotly                         plotly::plotly-5.10.0-py_0 --> conda-forge::plotly-5.14.1-pyhd8ed1ab_0 
  scikit-learn                         1.0.2-py39h4dfa638_0 --> 1.2.2-py39hd189fd4_1 


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...workin

## Obtendo arquivos de dados

Baixamos os arquivos caso o Notebook esteja sendo executado no ambiente do Google Colab. A variável de prefixo do caminho dos arquivos é definida de acordo.

In [7]:
if IN_COLAB:
    !git clone --depth=1 https://github.com/intrig-unicamp/hackathon5G.git hackathon5G

DATA_PATH_PREFIX = './hackathon5G/datasets' if IN_COLAB else '.'

## Importando dependências globais

In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics.pairwise import haversine_distances

from math import radians
import requests
import json
import random

pd.set_option('display.max_columns', None)

# Colunas

Para facilitar a compreensão das colunas disponíveis, pode-se agrupá-las logicamente da seguinte forma:

- **Identificação**: `NomeEntidade`, `NumEstacao`;

- **Rádio**: `Tecnologia`, `tipoTecnologia`, `FreqTxMHz`, `FreqRxMHz`, `DesignacaoEmissao`, `meioAcesso`, `Polarizacao`, `GanhoAntena`, `PotenciaTransmissorWatts`, `AnguloMeiaPotenciaAntena`, `FrenteCostaAntena`;

- **Posicionamento**: `Azimute`, `AlturaAntena`, `AnguloElevacao`;

- **Localização**: `Latitude`, `Longitude`, `EnderecoEstacao`, `EndComplemento`, `SiglaUf`, `CodMunicipio`;

- **Regulamentação e Licenciamento**: `NumFistel`, `NumFistelAssociado`, `NumAto`, `DataLicenciamento`, `DataPrimeiroLicenciamento`, `DataValidade`, `CodDebitoTFI`;

- _Outros_: `CodTipoClasseEstacao`, `ClassInfraFisica`, `CompartilhamentoInfraFisica`, `CodTipoAntena`, `CodEquipamentoAntena`, `CodEquipamentoTransmissor`, `NumRede`, `NumServico`, `NomeEntidadeAssociado`, `Status.state`, `_id`.

## Carregando os dados

In [5]:
MOSAICO_PATH = f'{DATA_PATH_PREFIX}/mosaico'

regions = [
    'são-paulo',
    'campinas',
    'rio-de-janeiro',
    'brasília',
]

ERBs = pd.concat([ pd.read_csv(f'{MOSAICO_PATH}/mosaico-erbs-{region}.zip', encoding='iso-8859-1', low_memory=False) for region in regions ], ignore_index=True, copy=True)
ERBs

,Status.state,NomeEntidade,NumFistel,NumServico,NumAto,NumEstacao,EnderecoEstacao,EndComplemento,SiglaUf,CodMunicipio,DesignacaoEmissao,Tecnologia,tipoTecnologia,meioAcesso,FreqTxMHz,FreqRxMHz,Azimute,CodTipoClasseEstacao,ClassInfraFisica,CompartilhamentoInfraFisica,CodTipoAntena,CodEquipamentoAntena,GanhoAntena,FrenteCostaAntena,AnguloMeiaPotenciaAntena,AnguloElevacao,Polarizacao,AlturaAntena,CodEquipamentoTransmissor,PotenciaTransmissorWatts,Latitude,Longitude,CodDebitoTFI,DataLicenciamento,DataPrimeiroLicenciamento,NumRede,_id,DataValidade,NumFistelAssociado,NomeEntidadeAssociado
0,LIC-LIC-01,TELEFONICA BRASIL S.A.,50409146447,10,17012008.0,122700,Rua Manoel Duque de Carvalho,NaN,SP,3550308,5M00G9W,WCDMA,NaN,NaN,2160.0,1970.0,230,FB,NaN,NaN,760,002000703518,14.9,25,70,6,X,41,018930701882,39.4,-23.71889,-46.68889,G,2021-12-24,1999-12-10,NaN,4d5c019f58e6a,2023-04-30,NaN,NaN
1,LIC-LIC-01,TELEFONICA BRASIL S.A.,50409146447,10,35312011.0,122700,Rua Manoel Duque de Carvalho,NaN,SP,3550308,200KG7W,GSM,NaN,NaN,1862.5,1767.5,360,FB,NaN,NaN,760,038801200762,17.8,25,65,5,X,39,006030301882,15.8,-23.71889,-46.68889,G,2021-12-24,1999-12-10,NaN,4d5c019f58e6b,2023-04-30,NaN,NaN
2,LIC-LIC-01,TELEFONICA BRASIL S.A.,50409146447,10,35312011.0,122700,Rua Manoel Duque de Carvalho,NaN,SP,3550308,200KG7W,GSM,NaN,NaN,1862.5,1767.5,130,FB,NaN,NaN,760,038801200762,17.8,25,65,5,X,39,006030301882,15.8,-23.71889,-46.68889,G,2021-12-24,1999-12-10,NaN,4d5c019f58e6c,2023-04-30,NaN,NaN
3,LIC-LIC-01,TELEFONICA BRASIL S.A.,50409146447,10,35312011.0,122700,Rua Manoel Duque de Carvalho,NaN,SP,3550308,200KG7W,GSM,NaN,NaN,1867.5,1772.5,130,FB,NaN,NaN,760,038801200762,17.8,25,65,5,X,39,006030301882,15.8,-23.71889,-46.68889,G,2021-12-24,1999-12-10,NaN,4d5c019f58e6d,2023-04-30,NaN,NaN
4,LIC-LIC-01,TELEFONICA BRASIL S.A.,50409146447,10,35312011.0,122700,Rua Manoel Duque de Carvalho,NaN,SP,3550308,200KG7W,GSM,NaN,NaN,1867.5,1772.5,240,FB,NaN,NaN,760,038801200762,17.8,25,65,5,X,39,006030301882,15.8,-23.71889,-46.68889,G,2021-12-24,1999-12-10,NaN,4d5c019f58e6e,2023-04-30,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1218865,LIC-LIC-01,TELEFONICA BRASIL S.A.,50409146366,10,35312011.0,1014783507,SHIS QI. 12 CONJUNTO 8/9,NaN,DF,5300108,10M0G7W,LTE,NaN,NaN,1855.0,1760.0,65.0,FB,NaN,NaN,760,NaN,13,20.0,63,0.0,X,4.0,083071803257,40,-15.83168,-47.86179,G,2023-03-21,2023-03-21,NaN,5ae855f89dc02039,2023-04-30,NaN,NaN
1218866,LIC-LIC-01,TELEFONICA BRASIL S.A.,50409146366,10,17012008.0,1014783507,SHIS QI. 12 CONJUNTO 8/9,NaN,DF,5300108,10M0G7W,LTE,NaN,NaN,2160.0,1970.0,335.0,FB,NaN,NaN,760,NaN,13.3,20.0,62,0.0,X,4.0,083071803257,40,-15.83168,-47.86179,G,2023-03-21,2023-03-21,NaN,5ae855f89dc0203a,2023-04-30,NaN,NaN
1218867,LIC-LIC-01,TELEFONICA BRASIL S.A.,50409146366,10,17012008.0,1014783507,SHIS QI. 12 CONJUNTO 8/9,NaN,DF,5300108,10M0G7W,LTE,NaN,NaN,2160.0,1970.0,65.0,FB,NaN,NaN,760,NaN,13.3,20.0,62,0.0,X,4.0,083071803257,40,-15.83168,-47.86179,G,2023-03-21,2023-03-21,NaN,5ae855f89dc0203b,2023-04-30,NaN,NaN
1218868,LIC-LIC-01,TELEFONICA BRASIL S.A.,50409146366,10,50852013.0,1014783507,SHIS QI. 12 CONJUNTO 8/9,NaN,DF,5300108,5M00G7W,LTE,NaN,NaN,2167.5,1977.5,335.0,FB,NaN,NaN,760,NaN,13.3,20.0,62,0.0,X,4.0,083071803257,40,-15.83168,-47.86179,G,2023-03-21,2023-03-21,NaN,5ae855f89dc0203c,2023-04-30,NaN,NaN


## Normalização dos valores

Na célula abaixo, identificamos os possíveis valores da coluna `Tecnologia`, em seguida, os normalizamos e criamos outro campo com o mapeamento para a geração da tecnologia (_e.g._ `LTE` → `4G`, `NR` → `5G`)

In [5]:
ERBs.Tecnologia.unique()

array(['WCDMA', 'GSM', 'LTE', 'NR', 'CDMA', nan, 'DMR', 'VHF', 'UHF',
       '0.0', 'TETRA', 'DMR - Tyer 2', 'WDCMA', 'NR ', 'LTE ', 'EDGE',
       'UMTS', 'Tetra'], dtype=object)

In [6]:
ERBs.replace({ 'Tecnologia': { 'NR ': 'NR', 'WDCMA': 'WCDMA' } }, inplace=True)

In [7]:
ERBs['Tecnologia_gen'] = ERBs.Tecnologia.map({ 'GSM': '2G', 'WCDMA': '3G', 'LTE': '4G', 'NR': '5G' })

A seguir, normalizamos os valores do azimute

In [8]:
ERBs.Azimute = ERBs.Azimute.str.replace(',', '.').replace('V', np.nan).astype(np.float64)

# Exploração

In [9]:
ERBs.NomeEntidade[(ERBs.NomeEntidade.str.contains('claro', case=False))].unique()

array(['CLARO S.A.'], dtype=object)

In [10]:
ERBs_SP_CLARO = ERBs[(ERBs.SiglaUf == 'SP') & (ERBs.NomeEntidade == 'CLARO S.A.')].copy()
ERBs_SP_CLARO

,Status.state,NomeEntidade,NumFistel,NumServico,NumAto,NumEstacao,EnderecoEstacao,EndComplemento,SiglaUf,CodMunicipio,DesignacaoEmissao,Tecnologia,tipoTecnologia,meioAcesso,FreqTxMHz,FreqRxMHz,Azimute,CodTipoClasseEstacao,ClassInfraFisica,CompartilhamentoInfraFisica,CodTipoAntena,CodEquipamentoAntena,GanhoAntena,FrenteCostaAntena,AnguloMeiaPotenciaAntena,AnguloElevacao,Polarizacao,AlturaAntena,CodEquipamentoTransmissor,PotenciaTransmissorWatts,Latitude,Longitude,CodDebitoTFI,DataLicenciamento,DataPrimeiroLicenciamento,NumRede,_id,DataValidade,NumFistelAssociado,NomeEntidadeAssociado,Tecnologia_gen
200,LIC-LIC-01,CLARO S.A.,50409105090,10,46662011.0,1246232,R GASPAR GONÇALVES RIBEIRO,0,SP,3550308,5M00G7W,WCDMA,NaN,NaN,885.0,840.0,20.0,FB,Greenfield,não,760,009371303519,14.89,30,65.28,6,X,35,039721705894,80,-23.72650,-46.70350,G,2021-07-14,2002-10-16,NaN,4d5c019f5b36e,2027-08-06,NaN,NaN,3G
201,LIC-LIC-01,CLARO S.A.,50409105090,10,46662011.0,1246232,R GASPAR GONÇALVES RIBEIRO,0,SP,3550308,5M00G7W,WCDMA,NaN,NaN,885.0,840.0,120.0,FB,Greenfield,não,760,009371303519,14.89,30,65.28,7,X,35,039721705894,80,-23.72650,-46.70350,G,2021-07-14,2002-10-16,NaN,4d5c019f5b36f,2027-08-06,NaN,NaN,3G
202,LIC-LIC-01,CLARO S.A.,50409105090,10,46662011.0,1246232,R GASPAR GONÇALVES RIBEIRO,0,SP,3550308,5M00G7W,WCDMA,NaN,NaN,885.0,840.0,260.0,FB,Greenfield,não,760,009371303519,14.89,30,65.28,9,X,35,039721705894,80,-23.72650,-46.70350,G,2021-07-14,2002-10-16,NaN,4d5c019f5b370,2027-08-06,NaN,NaN,3G
203,LIC-LIC-01,CLARO S.A.,50409105090,10,46662011.0,1246232,R GASPAR GONÇALVES RIBEIRO,0,SP,3550308,5M00G7W,WCDMA,NaN,NaN,885.0,840.0,20.0,FB,Greenfield,não,760,009371303519,14.89,30,65.28,6,X,35,039721705894,80,-23.72650,-46.70350,G,2021-07-14,2002-10-16,NaN,4d5c019f5b371,2027-08-06,NaN,NaN,3G
204,LIC-LIC-01,CLARO S.A.,50409105090,10,46662011.0,1246232,R GASPAR GONÇALVES RIBEIRO,0,SP,3550308,200KG7W,GSM,NaN,NaN,1825.0,1730.0,20.0,FB,Greenfield,não,760,027331703519,16.16,27,66.76,2,X,35,006091702519,79.43,-23.72650,-46.70350,G,2021-07-14,2002-10-16,NaN,4d5c019f5b372,2027-08-06,NaN,NaN,2G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
861976,LIC-LIC-01,CLARO S.A.,50409105090,10,105752021.0,1004778756,RUA MARIA MONTEIRO,ED PERO VAZ DE CAMINHA,SP,3509502,100MG7W,NR,SA-NSA,NaN,3350.0,3350.0,NaN,FB,NaN,NaN,760,NaN,23,25.0,13.5,-1.0,X,40.0,072212102519,200,-22.89167,-47.05333,G,2023-01-31,2017-07-25,NaN,6ae915f872a88a9b,2028-03-31,NaN,NaN,5G
861977,LIC-LIC-01,CLARO S.A.,50409105090,10,105752021.0,1004778756,RUA MARIA MONTEIRO,ED PERO VAZ DE CAMINHA,SP,3509502,100MG7W,NR,SA-NSA,NaN,3350.0,3350.0,NaN,FB,NaN,NaN,760,NaN,24.9,25.0,13.5,-2.0,X,40.0,102792102519,200,-22.89167,-47.05333,G,2023-01-31,2017-07-25,NaN,6ae915f872a88a9c,2028-03-31,NaN,NaN,5G
861978,LIC-LIC-01,CLARO S.A.,50409105090,10,105752021.0,1007421077,RUA LUIZ DE PAULA,NaN,SP,3509502,100MG7W,NR,SA-NSA,NaN,3350.0,3350.0,NaN,FB,NaN,NaN,760,NaN,23,25.0,13.5,0.0,X,50.0,072212102519,200,-22.86949,-46.96741,G,2023-01-31,2018-10-26,NaN,6ae915f872a88aca,2028-03-31,NaN,NaN,5G
861979,LIC-LIC-01,CLARO S.A.,50409105090,10,105752021.0,1007421077,RUA LUIZ DE PAULA,NaN,SP,3509502,100MG7W,NR,SA-NSA,NaN,3350.0,3350.0,NaN,FB,NaN,NaN,760,NaN,23,25.0,13.5,0.0,X,50.0,072212102519,200,-22.86949,-46.96741,G,2023-01-31,2018-10-26,NaN,6ae915f872a88acb,2028-03-31,NaN,NaN,5G


In [11]:
ERBs_SP_CLARO[ERBs_SP_CLARO.NumEstacao  == 1247468][['Tecnologia', 'tipoTecnologia']]

,Tecnologia,tipoTecnologia
217,WCDMA,NaN
218,GSM,NaN
219,GSM,NaN
220,GSM,NaN
221,LTE,NaN
222,LTE,NaN
223,LTE,NaN
224,WCDMA,NaN
225,WCDMA,NaN
226,WCDMA,NaN


In [12]:
# The Haversine (or great circle) distance is the angular distance
# between two points on the surface of a sphere. The first coordinate
# of each point is assumed to be the latitude, the second is the longitude,
# given in radians. The dimension of the data must be 2.

target = pd.DataFrame([[-23.555683, -46.662222]]).applymap(radians)
#target = pd.DataFrame([[-23.571094, -46.644239]]).applymap(radians)
distance = haversine_distances(ERBs_SP_CLARO[['Latitude', 'Longitude']].applymap(radians), target)
ERBs_SP_CLARO['distance'] = (distance * 6371000).round(3)  # multiply by Earth radius in meters
ERBs_SP_CLARO[(ERBs_SP_CLARO.distance < 2000)].groupby(['NumEstacao']).first().reset_index().sort_values('distance')

,NumEstacao,Status.state,NomeEntidade,NumFistel,NumServico,NumAto,EnderecoEstacao,EndComplemento,SiglaUf,CodMunicipio,DesignacaoEmissao,Tecnologia,tipoTecnologia,meioAcesso,FreqTxMHz,FreqRxMHz,Azimute,CodTipoClasseEstacao,ClassInfraFisica,CompartilhamentoInfraFisica,CodTipoAntena,CodEquipamentoAntena,GanhoAntena,FrenteCostaAntena,AnguloMeiaPotenciaAntena,AnguloElevacao,Polarizacao,AlturaAntena,CodEquipamentoTransmissor,PotenciaTransmissorWatts,Latitude,Longitude,CodDebitoTFI,DataLicenciamento,DataPrimeiroLicenciamento,NumRede,_id,DataValidade,NumFistelAssociado,NomeEntidadeAssociado,Tecnologia_gen,distance
58,683887904,LIC-LIC-01,CLARO S.A.,50409105090,10,46662011.0,R BELA CINTRA,COND. EDIFÍCIO QUARTIER OFFICE CENTER,SP,3550308,5M00G7W,WCDMA,NSA,NaN,885.0,840.0,5.0,FB,Rooftop,None,760,056001603518,14.00,23,62,0,X,50,039721705894,80,-23.555430,-46.661160,G,2022-08-03,2003-12-15,NaN,4d5c01a01349f,2027-08-06,NaN,None,3G,111.845
122,1000752124,LIC-LIC-01,CLARO S.A.,11021017965,175,280462002.0,Rua Bela Cintra 968 Consolação,None,SP,3550308,5M00G7W,None,None,NaN,882.8,837.8,350.0,FX,None,None,817,37751200762,14.00,35,70.6,2,X,51,010641001882,63.5,-23.555486,-46.661058,C,2020-03-28,2015-06-03,9334.0,265b3f0811f76659,2024-02-08,NaN,None,None,120.651
204,1001532705,LIC-LIC-01,CLARO S.A.,50403922577,45,626492006.0,Rua Bela Cintra 968 Consolação,None,SP,3550308,5M00G7W,None,None,NaN,885.0,840.0,90.0,FX,None,None,760,026591200762,14.40,33.0,66.37,2.00,X,51.0,010641001882,63.500,-23.555486,-46.661056,A,2015-05-28,2015-05-28,NaN,265b3f0811f45251,2028-11-29,NaN,None,None,120.894
253,1014365217,LIC-LIC-01,CLARO S.A.,11021017965,175,93272014.0,AV PAULISTA,EDIFICIO SAO LUIZ GONZAGA,SP,3550308,5M00G7W,WCDMA,None,NaN,2130.0,1940.0,0.0,FB,None,None,168,012110501403,4.70,0,360,0,V,2,006801103257,36.98,-23.556717,-46.661307,C,2022-10-20,2022-10-20,NaN,af8bc5f861901149,2027-07-02,NaN,None,3G,148.046
244,1011649664,LIC-LIC-01,CLARO S.A.,50409105090,10,34672011.0,AV PAULISTA,None,SP,3550308,5M00G7W,WCDMA,None,NaN,2140.0,1950.0,0.0,FB,Ran Sharing,não,168,000000INDOOR,4.70,0,360,0,V,3,006801103257,36.98,-23.556720,-46.661310,G,2022-06-23,2021-01-07,NaN,600599300b79d,2026-06-01,NaN,None,3G,148.113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,683895150,LIC-LIC-01,CLARO S.A.,50409105090,10,46662011.0,"R SÃO DOMINGOS, 148, BELA VISTA",None,SP,3550308,200KG7W,GSM,None,NaN,1830.0,1820.0,350.0,FB,Rooftop,sim,760,027331703519,16.16,27.0,66.76,2.0,X,32.4,006091702519,79.4,-23.551940,-46.643330,G,2020-07-27,2003-12-15,NaN,4d5c01a01426b,2027-08-06,NaN,None,2G,1970.140
247,1014365152,LIC-LIC-01,CLARO S.A.,11021017965,175,54202016.0,R CACONDE,0,SP,3550308,200KG7W / 5M00G7W / 10M0G7W / 5M00D7W / 10M0D7...,GSM,None,NaN,1825.0,1730.0,170.0,FB,None,None,760,056001603518,14.89,30,65.28,3,X,42.9,040952002519,60,-23.573400,-46.659700,C,2022-10-20,2022-10-20,NaN,af8bc5f86190112b,2027-07-02,NaN,None,2G,1986.739
136,1000752663,LIC-LIC-01,CLARO S.A.,11021017965,175,280462002.0,Rua Caconde 317 Jardim Paulista,None,SP,3550308,5M00D9W,None,None,NaN,882.8,837.8,180.0,FX,None,None,817,28661005344,13.50,28,58.9,16,X,44,008940701882,48.87,-23.573400,-46.659700,C,2020-03-28,2015-06-03,9352.0,265b3f0811f70b9b,2024-02-08,NaN,None,None,1986.739
172,1001314198,LIC-LIC-01,CLARO S.A.,50403922577,45,626492006.0,Rua Caconde 317 Jardim Paulista,None,SP,3550308,5M00D9W,None,None,NaN,885.0,840.0,180.0,FX,None,None,760,004620601705,13.50,28.0,58.90,16.00,X,44.0,008940701882,48.900,-23.573400,-46.659697,A,2015-05-26,2015-05-26,NaN,265b3f0811f407b5,2028-11-29,NaN,None,None,1986.776


In [13]:
df = ERBs_SP_CLARO[(ERBs_SP_CLARO.distance < 8000) & (ERBs_SP_CLARO.Tecnologia == 'NR')].sort_values('distance')
df.Latitude = df.Latitude.apply(lambda val: val + random.random() / 10**4)

In [14]:
df.Longitude = df.Longitude.apply(lambda val: val + random.random() / 10**4)

In [15]:
ERBs_SP_CLARO['Azimute_rad'] = ERBs_SP_CLARO.Azimute.apply(lambda val: val * np.pi/180)

In [16]:
ERBs_SP_CLARO[['Azimute', 'Azimute_rad']]

,Azimute,Azimute_rad
200,20.0,0.349066
201,120.0,2.094395
202,260.0,4.537856
203,20.0,0.349066
204,20.0,0.349066
...,...,...
861976,NaN,NaN
861977,NaN,NaN
861978,NaN,NaN
861979,NaN,NaN


In [17]:
import plotly.express as px

fig = px.scatter_mapbox(df, lat="Latitude", lon="Longitude", hover_name="NumEstacao", color="FreqTxMHz", #size="car_hours",
                  color_continuous_scale=px.colors.qualitative.Alphabet, size_max=20, zoom=13,  width=1000, height=1000)
fig.update_layout(
                        mapbox={
        #'style': "carto-darkmatter",
        #'style': "carto-positron",
        #'style': "open-street-map",
        'style': "stamen-toner",
        #'style': "stamen-watercolor",
        #'style': "white-bg",
                        },
                       )
#fig
#fig.show()

In [18]:
import plotly.express as px

df = ERBs_SP_CLARO[(ERBs_SP_CLARO.distance < 50000) & (ERBs_SP_CLARO.Tecnologia == 'NR')].sort_values('distance')
df.Latitude = df.Latitude.apply(lambda val: val + random.random() / 10**4)
df.Longitude = df.Longitude.apply(lambda val: val + random.random() / 10**4)

fig = px.scatter_mapbox(df, lat="Latitude", lon="Longitude", hover_name="NumEstacao", color="FreqTxMHz", #size="car_hours",
                  color_continuous_scale=px.colors.qualitative.Alphabet, size_max=20, zoom=13,  width=1000, height=1000)
fig.update_layout(
                        mapbox={
        #'style': "carto-darkmatter",
        #'style': "carto-positron",
        #'style': "open-street-map",
        'style': "stamen-toner",
        #'style': "stamen-watercolor",
        #'style': "white-bg",
                        },
                       )
fig.show()

In [19]:
ERBs[(ERBs.NumEstacao  == 683887904) & (ERBs.Tecnologia == 'LTE')][['Tecnologia', 'tipoTecnologia', 'DesignacaoEmissao', 'FreqTxMHz', 'FreqRxMHz']]
#9610

,Tecnologia,tipoTecnologia,DesignacaoEmissao,FreqTxMHz,FreqRxMHz
18214,LTE,NaN,20M0G7W,2640.0,2520.0
18215,LTE,NaN,20M0G7W,2640.0,2520.0
18216,LTE,NaN,20M0G7W,2640.0,2520.0
53925,LTE,NaN,5M00G7W,1825.0,1730.0
53926,LTE,NaN,5M00G7W,1825.0,1730.0
53927,LTE,NaN,5M00G7W,1825.0,1730.0
58251,LTE,NaN,10M0G7W,798.0,743.0
58252,LTE,NaN,10M0G7W,798.0,743.0
58253,LTE,NaN,10M0G7W,798.0,743.0
86658,LTE,NaN,5M00G7W,1825.0,1730.0


In [20]:
ERBs_SP_CLARO.groupby(['Tecnologia', 'tipoTecnologia'], dropna=False).first()[[]]

Empty DataFrame
Columns: []
Index: [(GSM, nan), (LTE, nan), (NR, NSA), (NR, SA), (NR, SA-NSA), (NR, nan), (WCDMA, nan), (nan, nan)]

In [21]:
ERBs.groupby(['Tecnologia_gen', 'tipoTecnologia', 'SiglaUf'], dropna=False).count()[['Status.state']]

Status.state
Tecnologia_gen tipoTecnologia SiglaUf              
2G             NaN            DF               7681
                              RJ              21799
                              SP              24627
3G             NaN            DF              18362
                              RJ              37605
                              SP              51062
4G             NaN            DF              26180
                              RJ              57513
                              SP              87937
5G             NSA            DF                924
                              RJ                  9
                              SP                591
               SA             DF                273
                              SP                 33
               SA-NSA         DF               3091
                              RJ               6661
                              SP              12851
               NaN            DF                 81
                              RJ                564
                              SP                598
NaN            0.0            SP                 12
               DMR Tyer 2     RJ                  7
                              SP                  2
               NaN            DF              48373
                              RJ             127766
                              SP             684268

In [22]:
ERBs[ERBs.CodMunicipio == 3550308]

,Status.state,NomeEntidade,NumFistel,NumServico,NumAto,NumEstacao,EnderecoEstacao,EndComplemento,SiglaUf,CodMunicipio,DesignacaoEmissao,Tecnologia,tipoTecnologia,meioAcesso,FreqTxMHz,FreqRxMHz,Azimute,CodTipoClasseEstacao,ClassInfraFisica,CompartilhamentoInfraFisica,CodTipoAntena,CodEquipamentoAntena,GanhoAntena,FrenteCostaAntena,AnguloMeiaPotenciaAntena,AnguloElevacao,Polarizacao,AlturaAntena,CodEquipamentoTransmissor,PotenciaTransmissorWatts,Latitude,Longitude,CodDebitoTFI,DataLicenciamento,DataPrimeiroLicenciamento,NumRede,_id,DataValidade,NumFistelAssociado,NomeEntidadeAssociado,Tecnologia_gen
0,LIC-LIC-01,TELEFONICA BRASIL S.A.,50409146447,10,17012008.0,122700,Rua Manoel Duque de Carvalho,NaN,SP,3550308,5M00G9W,WCDMA,NaN,NaN,2160.0,1970.0,230.0,FB,NaN,NaN,760,002000703518,14.9,25,70,6,X,41,018930701882,39.4,-23.718890,-46.688890,G,2021-12-24,1999-12-10,NaN,4d5c019f58e6a,2023-04-30,NaN,NaN,3G
1,LIC-LIC-01,TELEFONICA BRASIL S.A.,50409146447,10,35312011.0,122700,Rua Manoel Duque de Carvalho,NaN,SP,3550308,200KG7W,GSM,NaN,NaN,1862.5,1767.5,360.0,FB,NaN,NaN,760,038801200762,17.8,25,65,5,X,39,006030301882,15.8,-23.718890,-46.688890,G,2021-12-24,1999-12-10,NaN,4d5c019f58e6b,2023-04-30,NaN,NaN,2G
2,LIC-LIC-01,TELEFONICA BRASIL S.A.,50409146447,10,35312011.0,122700,Rua Manoel Duque de Carvalho,NaN,SP,3550308,200KG7W,GSM,NaN,NaN,1862.5,1767.5,130.0,FB,NaN,NaN,760,038801200762,17.8,25,65,5,X,39,006030301882,15.8,-23.718890,-46.688890,G,2021-12-24,1999-12-10,NaN,4d5c019f58e6c,2023-04-30,NaN,NaN,2G
3,LIC-LIC-01,TELEFONICA BRASIL S.A.,50409146447,10,35312011.0,122700,Rua Manoel Duque de Carvalho,NaN,SP,3550308,200KG7W,GSM,NaN,NaN,1867.5,1772.5,130.0,FB,NaN,NaN,760,038801200762,17.8,25,65,5,X,39,006030301882,15.8,-23.718890,-46.688890,G,2021-12-24,1999-12-10,NaN,4d5c019f58e6d,2023-04-30,NaN,NaN,2G
4,LIC-LIC-01,TELEFONICA BRASIL S.A.,50409146447,10,35312011.0,122700,Rua Manoel Duque de Carvalho,NaN,SP,3550308,200KG7W,GSM,NaN,NaN,1867.5,1772.5,240.0,FB,NaN,NaN,760,038801200762,17.8,25,65,5,X,39,006030301882,15.8,-23.718890,-46.688890,G,2021-12-24,1999-12-10,NaN,4d5c019f58e6e,2023-04-30,NaN,NaN,2G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726315,LIC-LIC-01,TELEFONICA BRASIL S.A.,50409146447,10,35312011.0,1000598664,RUA JOSUÉ CARMINO BRUNO,NaN,SP,3550308,5M00G7W,LTE,NaN,NaN,1862.5,1767.5,120.0,FB,NaN,NaN,760,56001603518,15.0,25,56,2,X,38,040952002519,40,-23.585382,-46.512821,G,2023-01-23,2014-11-24,NaN,762525f8473607c1,2029-12-08,NaN,NaN,4G
726316,LIC-LIC-01,TELEFONICA BRASIL S.A.,50409146447,10,35312011.0,1000598664,RUA JOSUÉ CARMINO BRUNO,NaN,SP,3550308,5M00G7W,LTE,NaN,NaN,1862.5,1767.5,240.0,FB,NaN,NaN,760,56001603518,15.0,25,56,2,X,38,040952002519,40,-23.585382,-46.512821,G,2023-01-23,2014-11-24,NaN,762525f8473607c2,2029-12-08,NaN,NaN,4G
726317,LIC-LIC-01,TELEFONICA BRASIL S.A.,50409146447,10,41662008.0,1000598664,RUA JOSUÉ CARMINO BRUNO,NaN,SP,3550308,5M00G7W,WCDMA,NaN,NaN,874.5,829.5,20.0,FB,NaN,NaN,760,56001603518,14.5,25,65,2,X,38,000081201882,40,-23.585382,-46.512821,G,2023-01-23,2014-11-24,NaN,762525f8473607c3,2029-12-08,NaN,NaN,3G
726318,LIC-LIC-01,TELEFONICA BRASIL S.A.,50409146447,10,41662008.0,1000598664,RUA JOSUÉ CARMINO BRUNO,NaN,SP,3550308,5M00G7W,WCDMA,NaN,NaN,874.5,829.5,120.0,FB,NaN,NaN,760,56001603518,14.5,25,65,2,X,38,000081201882,40,-23.585382,-46.512821,G,2023-01-23,2014-11-24,NaN,762525f8473607c4,2029-12-08,NaN,NaN,3G


In [23]:
def getCodUF(siglaUf):
    ufList = [{"CodUF":"11","SiglaUF":"RO"},{"CodUF":"14","SiglaUF":"RR"},{"CodUF":"16","SiglaUF":"AP"},{"CodUF":"17","SiglaUF":"TO"},{"CodUF":"22","SiglaUF":"PI"},{"CodUF":"24","SiglaUF":"RN"},{"CodUF":"26","SiglaUF":"PE"},{"CodUF":"29","SiglaUF":"BA"},{"CodUF":"33","SiglaUF":"RJ"},{"CodUF":"42","SiglaUF":"SC"},{"CodUF":"51","SiglaUF":"MT"},{"CodUF":"27","SiglaUF":"AL"},{"CodUF":"31","SiglaUF":"MG"},{"CodUF":"41","SiglaUF":"PR"},{"CodUF":"50","SiglaUF":"MS"},{"CodUF":"53","SiglaUF":"DF"},{"CodUF":"12","SiglaUF":"AC"},{"CodUF":"13","SiglaUF":"AM"},{"CodUF":"15","SiglaUF":"PA"},{"CodUF":"21","SiglaUF":"MA"},{"CodUF":"23","SiglaUF":"CE"},{"CodUF":"25","SiglaUF":"PB"},{"CodUF":"28","SiglaUF":"SE"},{"CodUF":"32","SiglaUF":"ES"},{"CodUF":"35","SiglaUF":"SP"},{"CodUF":"43","SiglaUF":"RS"},{"CodUF":"52","SiglaUF":"GO"}]
    for uf in ufList:
        if siglaUf == uf['SiglaUF']:
            return uf['CodUF']

    return None

In [24]:
def getUFMunicipios(siglaUf):
    CodUF = getCodUF(siglaUf)
    url = f'https://sistemas.anatel.gov.br/se/eApp/forms/b/jf_getMunicipios.php?CodUF={CodUF}'
    response = requests.get(url).text
    return json.loads(response)

In [25]:
def getMunicipioId(siglaUf, municipioName):
    municipiosList = getUFMunicipios(siglaUf)
    for mun in municipiosList:
        if municipioName == mun[0]:
            return int(mun[1])
    return None

In [26]:
#getCodUF('SP'), getMunicipioId('SP', 'São Paulo')
getUFMunicipios('DF'), getCodUF('DF'), getMunicipioId('DF', 'Brasília')

([['Todos os Municipios', '5300000'],
  ['Brasília', '5300108', -15.7989, -47.8667]],
 '53',
 5300108)

In [27]:
df = ERBs_SP_CLARO[(ERBs_SP_CLARO.distance < 2000) & (ERBs_SP_CLARO.Tecnologia == 'NR')]
#df.Latitude = df.Latitude.apply(lambda val: val + random.random() / 10**4)
#df.Longitude = df.Longitude.apply(lambda val: val + random.random() / 10**4)

In [28]:
import plotly.graph_objs as go
from tqdm import tqdm

# Load data from CSV file
data = df

# Create a scatter mapbox trace with arrow markers
fig = go.Figure()

# Add arrow markers for each data point
for _, row in tqdm(data.iterrows()):
    size = .0001
    random_pos = hash(row.FreqTxMHz) / 10**13.7
    random_rot = random.random() / 10**1
    if row['Azimute'] == 0:
        fig.add_trace(go.Scattermapbox(
            lat=data['Latitude'] + random_pos,
            lon=data['Longitude'] + random_pos,
            mode='markers',
            marker=dict(size=7, color='red', symbol='circle'),
        ))
    else:
        fig.add_trace(go.Scattermapbox(
            lat = [row['Latitude'] + random_pos, row['Latitude'] + size * np.cos(row['Azimute_rad'] + random_rot) + random_pos],
            lon = [row['Longitude'] + random_pos, row['Longitude'] + size * np.sin(row['Azimute_rad'] + random_rot) + random_pos],
            mode = 'lines',
            line = dict(width = 1.5, color = 'blue'),
        ))

537it [00:00, 1506.41it/s]


In [29]:
# Set mapbox style and layout
fig.update_layout(
    mapbox=dict(
        #style= "carto-darkmatter",
        #style= "carto-positron",
        #style= "open-street-map",
        style= "stamen-toner",
        #style= "stamen-watercolor",
        #style= "white-bg",
        center=dict(lat=data['Latitude'].iloc[0], lon=data['Longitude'].iloc[0]),
        zoom=18),
    margin=dict(l=0, r=0, t=0, b=0),
    showlegend=False,
        width=1000, height=1000,
)

# Show the figure
fig.show()